In [1]:
import pandas as pd
import numpy as np

In [2]:
gps_names = ['db_key', 'time', 'latitude', 'longitude', 'altitude', 'speed', 'horizontal_accuracy',
             'horizontal_dop', 'vertical_accuracy', 'vertical_dop', 'speed_accuracy', 'time_since_gps_boot']
gps = pd.read_csv('uncompressed/mdcdb_1144/gps.csv', names=gps_names, sep='\t', 
                  index_col=False, na_values='\\N')

In [3]:
gps.shape

(11077061, 12)

In [4]:
gps.head()

,db_key,time,latitude,longitude,altitude,speed,horizontal_accuracy,horizontal_dop,vertical_accuracy,vertical_dop,speed_accuracy,time_since_gps_boot
0,79237733,1252269937,7.200,46.100,1269.0,NaN,NaN,3.56,146.487274,1.0,1.0,155.0
1,79237735,1252269947,7.200,46.100,1269.0,NaN,NaN,3.57,92.526222,1.0,1.0,165.0
2,79237739,1252269957,7.200,46.100,1269.0,NaN,NaN,3.58,131.370636,1.0,4.0,175.0
3,80235202,1252932684,6.565,46.524,460.0,NaN,NaN,5.00,359.116455,1.0,18.0,277.0
4,80235205,1252932694,6.565,46.524,460.0,NaN,NaN,5.01,174.380981,1.0,6.0,287.0


In [5]:
gps = gps[['db_key', 'time', 'latitude', 'longitude']]
gps.head()

,db_key,time,latitude,longitude
0,79237733,1252269937,7.200,46.100
1,79237735,1252269947,7.200,46.100
2,79237739,1252269957,7.200,46.100
3,80235202,1252932684,6.565,46.524
4,80235205,1252932694,6.565,46.524


In [6]:
gps.index.dtype

dtype('int64')

In [7]:
records_names = ['db_key', 'userid', 'time', 'tz', 'type']
records = pd.read_csv('uncompressed/mdcdb_1144/records.csv', names=records_names, sep='\t',
                      index_col=False, na_values='\\N')

In [8]:
records.shape

(251224015, 5)

In [9]:
records.head()

,db_key,userid,time,tz,type
0,235597273,5988,1274969244,-7200,gps
1,214255312,6178,1271335119,-7200,gps
2,229985352,6054,1274736793,-7200,gps
3,238957887,6199,1275825737,-7200,gps
4,336354268,6085,1290635679,-3600,gps


In [10]:
# Attempt to merge records and gps by setting both their indicies to db_key and then glueing them together
#    POSSIBLE PROBLEM: There appear to be duplicate db_keys in records, which is odd

df = pd.merge(gps, records, on='db_key')
df.head()

,db_key,time_x,latitude,longitude,userid,time_y,tz,type
0,79237733,1252269937,7.200,46.100,5542,1252269937,-7200,gps
1,79237733,1252269937,7.200,46.100,5542,1252269937,-7200,gps
2,79237735,1252269947,7.200,46.100,5542,1252269947,-7200,gps
3,79237739,1252269957,7.200,46.100,5542,1252269957,-7200,gps
4,80235202,1252932684,6.565,46.524,5921,1252932684,-7200,gps


In [11]:
del gps
del records

In [12]:
(df['time_x'] == df['time_y']).value_counts()

True     13291097
False      387521
dtype: int64

In [13]:
df[df['time_x'] != df['time_y']].head()

,db_key,time_x,latitude,longitude,userid,time_y,tz,type
57,78685791,1251786507,6.639824,46.522144,5479,1251786508,-3600,gps
82,78685839,1251786825,6.620485,46.523500,5479,1251786826,-3600,gps
152,78685924,1251787413,6.574187,46.522545,5479,1251787414,-3600,gps
198,78699685,1251786025,6.646970,46.509037,5578,1251786026,-7200,gps
373,78702477,1251823613,6.653587,46.509215,5578,1251823614,-7200,gps


In [14]:
(df['time_x'] - df['time_y']).abs().value_counts().sort_index()

0     13291097
1       149845
2        34344
3        33622
4        32310
5        26685
6        20663
7        17447
8        18950
9        33374
10       19745
11          65
12          12
13           7
14          23
15          67
16          59
17          34
18          31
19          35
20          31
21          37
22          36
23          29
24          29
25          21
26           3
27           1
28           3
29           3
30           1
34           1
35           3
39           1
57           1
59           1
65           1
66           1
dtype: int64

In [15]:
df['time'] = df['time_y']
df.drop(['time_x', 'time_y'], axis=1, inplace=True)

df.head()

,db_key,latitude,longitude,userid,tz,type,time
0,79237733,7.200,46.100,5542,-7200,gps,1252269937
1,79237733,7.200,46.100,5542,-7200,gps,1252269937
2,79237735,7.200,46.100,5542,-7200,gps,1252269947
3,79237739,7.200,46.100,5542,-7200,gps,1252269957
4,80235202,6.565,46.524,5921,-7200,gps,1252932684


In [16]:
df.drop('db_key', axis=1, inplace=True)
df.head()

,latitude,longitude,userid,tz,type,time
0,7.200,46.100,5542,-7200,gps,1252269937
1,7.200,46.100,5542,-7200,gps,1252269937
2,7.200,46.100,5542,-7200,gps,1252269947
3,7.200,46.100,5542,-7200,gps,1252269957
4,6.565,46.524,5921,-7200,gps,1252932684


In [17]:
df['time'] = pd.to_datetime(df['time'], unit='s')
df.head()

,latitude,longitude,userid,tz,type,time
0,7.200,46.100,5542,-7200,gps,2009-09-06 20:45:37
1,7.200,46.100,5542,-7200,gps,2009-09-06 20:45:37
2,7.200,46.100,5542,-7200,gps,2009-09-06 20:45:47
3,7.200,46.100,5542,-7200,gps,2009-09-06 20:45:57
4,6.565,46.524,5921,-7200,gps,2009-09-14 12:51:24


In [18]:
# Change the way the dataset is indexed

df = df.set_index(['userid', 'time'])
df.sort_index(inplace=True)

df.head(10)

latitude  longitude    tz type
userid time                                               
5448   2009-10-31 15:55:40  6.632476  46.520321 -7200  gps
       2009-10-31 16:34:37  6.592562  46.523943 -7200  gps
       2009-10-31 16:34:46  6.594179  46.524187 -7200  gps
       2009-10-31 16:34:46  6.594179  46.524187 -7200  gps
       2009-10-31 16:34:57  6.596068  46.523909 -7200  gps
       2009-10-31 16:34:57  6.596068  46.523909 -7200  gps
       2009-10-31 16:34:57  6.596068  46.523909 -7200  gps
       2009-10-31 16:35:06  6.598078  46.523439 -7200  gps
       2009-10-31 16:35:16  6.600007  46.523223 -7200  gps
       2009-10-31 16:35:16  6.600007  46.523223 -7200  gps

In [19]:
# Save the dataset to .pkl

df.to_pickle('mdc.pkl')